# TradeAgreements for AX 7 Currencies

* shift + enter to run one cell of code
* 7 currencies (or less if desired)
* after new folder created it opens automatically, save all productCatalogExpanded files there
* head() to see the first rows
* .shape to see the document shape
* Price Group naming conv. max 20 char

In [1]:
import pandas as pd

In [2]:
import os

In [3]:
import re

In [4]:
import xlsxwriter

In [5]:
from datetime import datetime

In [6]:
from datetime import timedelta

In [7]:
from datetime import date

In [8]:
from pathlib import Path

In [9]:
import numpy as np

In [10]:
import webbrowser

In [11]:
import requests

In [12]:
os.getcwd()

'C:\\Users\\hahafonova'

## Create new version directory 
* Naming convension for version folders: TAs_MONTH YEAR
* Naming convension for Trade Agreements: TA_yearCURR-NEW
* Naming convension for Price Group Name and Account selection: yearCURR-NEW

In [13]:
getmonth = str.upper(datetime.now().strftime('%b'))

In [14]:
getyear = datetime.now().strftime('%Y')

In [15]:
name_dir = "TAs_{month} {year}"

In [16]:
directory = name_dir.format(year=getyear, month=getmonth)

In [17]:
parent_dir = "R:\doc\Kunden Produktlisten\zz_TAs_Prices"

In [18]:
dir_path = os.path.join(parent_dir, directory)

In [19]:
isExist = os.path.exists(dir_path)

In [20]:
if not isExist:
    os.makedirs(dir_path)
    print("Directory '%s' created" %directory)
else:
    print("Directory '%s' exists" %directory)

Directory 'TAs_MAR 2025' created


In [21]:
webbrowser.open(os.path.realpath(dir_path)) # opens the directory of current working file

True

## Import productCatalogExpanded from 
* Currency
* All
* No
* No
* Download all and copy than all docs from Downloads folder ! **no renaming required ! folder cleans itself !**

In [22]:
# BELOW loop for all currencies!! and renaming

In [23]:
url = "http://internal.thorlabs.com/productCatalogExpanded.cfm"

In [24]:
parameter = {'curr': ["Dollar", "Euro", "GBPound", "Krona", "JPY", "RMB", "BRL"], 'prodType':"All", 'superceded':"no", 'GM': '', 'PartList':'*'}

In [25]:
for curr_value in parameter['curr']:
    params = {'curr': curr_value, 'prodType':"All", 'superceded':"no", 'GM': '', 'PartList':'*'}

    response = requests.post(url, data=params)

    if response.status_code == 200:
        download_link = "http://internal.thorlabs.com/ExcelDownloads/productCatalogExpanded.xls"  

        excel_file = requests.get(download_link)

        with open("{}\productCatalogExpanded{}.xls".format(dir_path, curr_value), 'wb') as f:
            f.write(excel_file.content)
            print(curr_value, "Excel file downloaded successfully")
    else:
        print(f"Form submission failed with status code: {response.status_code}")

Dollar Excel file downloaded successfully
Euro Excel file downloaded successfully
GBPound Excel file downloaded successfully
Krona Excel file downloaded successfully
JPY Excel file downloaded successfully
RMB Excel file downloaded successfully
BRL Excel file downloaded successfully


In [26]:
def check_currency(file):
    '''Check column CURR (currency in the file)
    :param file: productCatalogExpnded path
    :return curr: checkes column CURR, makes list from set of values, extracts value
    :exception: delete files with bad path
    >>> check_currency("_")
    ['Dollar']'''
    try:
        df = pd.read_excel(file)
        curr = list(set(df['CURR'].values))[0]
        curr = curr.replace('Dollar', 'USD').replace('Euro', 'EUR').replace('GBPound', 'GBP').replace('Krona', 'SEK')
        return curr
    except (IOError, ValueError):
        os.remove(file)

In [27]:
print(os.listdir(dir_path)) # list of files in newly created folder

['productCatalogExpandedBRL.xls', 'productCatalogExpandedDollar.xls', 'productCatalogExpandedEuro.xls', 'productCatalogExpandedGBPound.xls', 'productCatalogExpandedJPY.xls', 'productCatalogExpandedKrona.xls', 'productCatalogExpandedRMB.xls']


In [28]:
class TradeAgreement(pd.DataFrame):
    '''Creates instance according to naming convention.'''
    def __init__(self, df, *args, **kwargs):
        pd.DataFrame.__init__(self, df, *args, **kwargs)
        print("\n{}{}-NEW".format(getyear, list(set(df['CURR'].values))[0]))
        
    def select(self): # selects relevant columns
        self = self[['PARTNUM', 'QTYAMOUNT', 'VOLUMEPRICE', 'CURR']]
        self['charity'] = self.apply(lambda _: '', axis=1)
        self['charity'] = self[['PARTNUM', 'QTYAMOUNT']].agg(''.join, axis=1)
        self = self.drop_duplicates(subset=['charity'])
        self = self[self['VOLUMEPRICE'] != 0]
        self['VOLUMEPRICE'] = self['VOLUMEPRICE'].round(2)
        return self
        
    def set_columns(self): # creates columns for TAs and inserts static values
        self['Price/discount journal number'] = self.apply(lambda _: '', axis=1)
        self['Line number'] = self.apply(lambda _: '', axis=1)
        self.index.name = 'Line number'
        self['Relation'] = 'Price (sales)'
        self['Account code'] = 'Group'
        self['Account selection'] = self.apply(lambda _: '', axis=1)
        self['Item code'] = 'Table'
        self['Item relation'] = self.apply(lambda _: '', axis=1)
        self['Configuration'] = self.apply(lambda _: '', axis=1)
        self['Site'] = self.apply(lambda _: '', axis=1)
        self['Warehouse'] = self.apply(lambda _: '', axis=1)
        self['Location'] = self.apply(lambda _: '', axis=1)
        self['From'] = self['QTYAMOUNT']
        self['To'] = self.apply(lambda _: '', axis=1)
        self['Unit'] = 'Pcs'
        self['Amount in currency'] = self['VOLUMEPRICE']
        self['From date'] = datetime.today().strftime('%d.%m.%Y')
        self['To date'] = self.apply(lambda _: '', axis=1)
        self['Currency'] = self.apply(lambda _: '', axis=1)
        self['Price charges'] = '0'
        self['Price unit'] = '1'
        self['Discount percentage 1'] = '0'
        self['Discount percentage 2'] = '0'
        self['Lead time'] = '0'
        self['Working days'] = 'No'
        self['Disregard lead time'] = 'Yes'
        self['Include generic currency'] = 'No'
        self['Find next'] = 'Yes'
        return self
        
    def change_values(self): # changes dynamic version values
        self['Item relation'] = self['PARTNUM'].map(str)    
        todate = timedelta(days=1095)
        TA_expire = datetime.today() + todate
        self['To date'] = TA_expire.strftime('%d.%m.%Y')
        curr = str(list(set(df['CURR'].values))[0])
        curr = curr.replace('Dollar', 'USD').replace('Euro', 'EUR').replace('GBPound', 'GBP').replace('Krona', 'SEK')
        self['Account selection'] = "{}{}-NEW".format(getyear, curr) #autamated Price Group NAME
        self['Currency'] = curr
        return self
        
    def change_index(self): # changes index to recatlulate Line number
        self = self.reset_index(drop=True)
        self.index = self.index + 1
        self['Line number'] = self.index
        return self
        
    def drop_col(self): # drops excess columns
        cols = ['Price/discount journal number', 'Line number', 'Relation', 'Account code', 'Account selection', 
                'Item code', 'Item relation', 'Configuration', 'Site', 'Warehouse', 'Location', 'From', 'To', 'Unit',
                'Amount in currency', 'From date', 'To date', 'Currency', 'Price charges', 'Price unit', 
                'Discount percentage 1', 'Discount percentage 2', 'Lead time', 'Working days', 'Disregard lead time', 
                'Include generic currency', 'Find next']
        for column in self.columns:
            if column not in cols:
                self = self.drop([column], axis=1)
                print(column, '\t\tdeleted')
        return self
         
    def save_ta(self):
        curr = str(list(set(df['CURR'].values))[0])
        curr = curr.replace('Dollar', 'USD').replace('Euro', 'EUR').replace('GBPound', 'GBP').replace('Krona', 'SEK')
        file_suffix = "{}{}-NEW".format(getyear, curr)
        try:
            self.to_excel(f'{dir_path}\TA_{file_suffix}.xlsx', 
                index=False)
        except FileExistsError:
            print('File exists')

In [29]:
pd.options.mode.chained_assignment = None  # default='warn', changed for error canceling

In [30]:
for files in os.listdir(dir_path):
    if files.endswith('.xls'): 
        df = pd.read_excel("{}\{}".format(dir_path, files), engine='xlrd', dtype={'PARTNUM':str, 'QTYAMOUNT':str})
        ta = TradeAgreement(df)
        ta = TradeAgreement.select(ta)
        ta = TradeAgreement.set_columns(ta)
        ta = TradeAgreement.change_values(ta)
        ta = TradeAgreement.change_index(ta)
        ta = TradeAgreement.drop_col(ta)
        TradeAgreement.save_ta(ta)
    else:
        continue


2025BRL-NEW
PARTNUM 		deleted
QTYAMOUNT 		deleted
VOLUMEPRICE 		deleted
CURR 		deleted
charity 		deleted

2025Dollar-NEW
PARTNUM 		deleted
QTYAMOUNT 		deleted
VOLUMEPRICE 		deleted
CURR 		deleted
charity 		deleted

2025Euro-NEW
PARTNUM 		deleted
QTYAMOUNT 		deleted
VOLUMEPRICE 		deleted
CURR 		deleted
charity 		deleted

2025GBPound-NEW
PARTNUM 		deleted
QTYAMOUNT 		deleted
VOLUMEPRICE 		deleted
CURR 		deleted
charity 		deleted

2025JPY-NEW
PARTNUM 		deleted
QTYAMOUNT 		deleted
VOLUMEPRICE 		deleted
CURR 		deleted
charity 		deleted

2025Krona-NEW
PARTNUM 		deleted
QTYAMOUNT 		deleted
VOLUMEPRICE 		deleted
CURR 		deleted
charity 		deleted

2025RMB-NEW
PARTNUM 		deleted
QTYAMOUNT 		deleted
VOLUMEPRICE 		deleted
CURR 		deleted
charity 		deleted


## Semi-Automates Test 

In [31]:
user = os.getcwd()

In [32]:
user = user.replace('C:\\Users\\', '')

In [33]:
user

'hahafonova'

#### TO DO:
* sample size 3-5 per file
* volume price items
* loop: item, volume, price

In [34]:
class SampleData(pd.DataFrame):
    '''Creates sample.'''
    def __init__(self, df, *args, **kwargs):
        pd.DataFrame.__init__(self, df, *args, **kwargs)
        
    def select(self):
        self = self[['Item relation', 'From', 'Amount in currency', 'Currency']]
        self = self[self['From'] == 1]
        self = self.iloc[np.random.randint(low=0, high=int(self.shape[0]), size=int(self.shape[0]/600))]
        return self
        
    def check_price(self):  
        for i in self['Item relation']:
            webbrowser.open_new('https://www.thorlabs.com/thorproduct.cfm?partnumber={}'.format(i))
            print('\nItem: ', i)
            print('Currency: ', self[self['Item relation']==i]['Currency'].item())
            print('Vol.: ', self[self['Item relation']==i]['From'].item())
            print('Price: ', self[self['Item relation']==i]['Amount in currency'].item())
            if str(input('Is it correct on the page (y/n)? ')) == 'y':
                continue
            else:
                print('Error! correct the file!')
                break

In [35]:
dir_path

'R:\\doc\\Kunden Produktlisten\\zz_TAs_Prices\\TAs_MAR 2025'

In [36]:
print(os.listdir(dir_path))

['productCatalogExpandedBRL.xls', 'productCatalogExpandedDollar.xls', 'productCatalogExpandedEuro.xls', 'productCatalogExpandedGBPound.xls', 'productCatalogExpandedJPY.xls', 'productCatalogExpandedKrona.xls', 'productCatalogExpandedRMB.xls', 'TA_2025BRL-NEW.xlsx', 'TA_2025EUR-NEW.xlsx', 'TA_2025GBP-NEW.xlsx', 'TA_2025JPY-NEW.xlsx', 'TA_2025RMB-NEW.xlsx', 'TA_2025SEK-NEW.xlsx', 'TA_2025USD-NEW.xlsx']


In [37]:
# print [y] enter to continue 3 items per file or [n] enter to stop cheking current file

for files in os.listdir(dir_path):
    if files.endswith('.xlsx'): 
        df = pd.read_excel("{}\{}".format(dir_path, files))
        sample = SampleData(df)
        sample = SampleData.select(sample)
        SampleData.check_price(sample)
        


Item:  TED200C
Currency:  BRL
Vol.:  1
Price:  15257.58
Is it correct on the page (y/n)? y

Item:  PFP6090-7
Currency:  BRL
Vol.:  1
Price:  27336.45
Is it correct on the page (y/n)? n
Error! correct the file!

Item:  CFCS11-A
Currency:  EUR
Vol.:  1
Price:  287.39
Is it correct on the page (y/n)? n
Error! correct the file!

Item:  PLE152
Currency:  GBP
Vol.:  1
Price:  57.77
Is it correct on the page (y/n)? n
Error! correct the file!

Item:  POLARIS-K1T1
Currency:  JPY
Vol.:  1
Price:  34177.0
Is it correct on the page (y/n)? n
Error! correct the file!

Item:  HCA3
Currency:  RMB
Vol.:  1
Price:  497.43
Is it correct on the page (y/n)? n
Error! correct the file!

Item:  IO-K-2000
Currency:  SEK
Vol.:  1
Price:  36564.0
Is it correct on the page (y/n)? n
Error! correct the file!

Item:  OSW12-830-SM
Currency:  USD
Vol.:  1
Price:  1148.72
Is it correct on the page (y/n)? n
Error! correct the file!


In [38]:
import win32com.client as win32

In [39]:
text = '''
Dear colleagues,

Please find Trade Agreements in attachment.

The TA contain NEW prices for entities in Europe and THO. Please upload it as follows:
–	UK0: {year}GBP-NEW
–	FRA: {year}EUR-NEW
–	MUC: {year}EUR-NEW
–	SWE: {year}EUR-NEW and 2023SEK-NEW
–	THO: {year}USD-NEW


Please upload it to AX. 

Best regards,
{close}

'''.format(year=date.today().strftime('%Y'), close='THIS MESSAGE WAS GENERATED AND SENT AUTOMATICALLY')

In [40]:
signature = '''i.A.
{name}
{position}

Phone: +49-8131-5956-4024-9
{user}@thorlabs.com 


Thorlabs GmbH ۰ Münchner Weg 1 ۰ 85232 Bergkirchen ۰ Germany
GM: Dr. Julien Vigroux  ۰ HRB: 85345 ۰ Amtsgericht München



DS-GVO: Wir erheben personenbezogene Daten von Ihnen, die elementar für die Geschäftsabwicklung sind. Aus diesem Grund sind wir nach Artikel 13 DS-GVO verpflichtet, Sie darüber zu informieren, wie Ihre Daten verarbeitet und gespeichert werden. Bitte nutzen Sie dazu folgenden Link: https://www.thorlabs.us/privacy.cfm
GDPR: We collect personal data from you that is fundamental to the business transaction. For this reason, we are obliged under Article 13 GDPR to inform you about how your data is processed and stored. Please refer to the following link for details: https://www.thorlabs.us/privacy.cfm
'''.format(name=str(input('paste your name: ')), position=str(input('paste your postition: ')), user=os.getcwd().replace('C:\\Users\\', ''))

paste your name: hanna
paste your postition: test


In [43]:
os.listdir(dir_path)

['productCatalogExpandedBRL.xls',
 'productCatalogExpandedDollar.xls',
 'productCatalogExpandedEuro.xls',
 'productCatalogExpandedGBPound.xls',
 'productCatalogExpandedJPY.xls',
 'productCatalogExpandedKrona.xls',
 'productCatalogExpandedRMB.xls',
 'TA_2025BRL-NEW.xlsx',
 'TA_2025EUR-NEW.xlsx',
 'TA_2025GBP-NEW.xlsx',
 'TA_2025JPY-NEW.xlsx',
 'TA_2025RMB-NEW.xlsx',
 'TA_2025SEK-NEW.xlsx',
 'TA_2025USD-NEW.xlsx']

In [46]:
mail = ['axapta@thorlabs.com', 'phardtke@thorlabs.com', 'EDI-europe@thorlabs.com', 'cbozzetti@thorlabs.com']
olApp = win32.Dispatch('Outlook.Application')
olNS = olApp.GetNameSpace('MAPI')
mailItem = olApp.CreateItem(0)
mailItem.Subject = 'New Thorlabs Trade Agreements {} {}'.format(getmonth, getyear)
mailItem.BodyFormat = 1
mailItem._oleobj_.Invoke(*(64209, 0, 8, 0, olNS.Accounts.Item('EDI-europe@thorlabs.com')))

mailItem.Body = (text + '\n\n\n'+ signature)
mailItem.To = '; '.join(mail)
mailItem.CC = 'EDI-europe@thorlabs.com'

for e in os.listdir(dir_path):
    if e.startswith('TA'):
        full_path = os.path.join(dir_path, e)
        mailItem.Attachments.Add(full_path)
        print(e, 'attached')
    else:
        continue
print('DONE')

mailItem.Display()

TA_2025BRL-NEW.xlsx attached
TA_2025EUR-NEW.xlsx attached
TA_2025GBP-NEW.xlsx attached
TA_2025JPY-NEW.xlsx attached
TA_2025RMB-NEW.xlsx attached
TA_2025SEK-NEW.xlsx attached
TA_2025USD-NEW.xlsx attached
DONE
